In [2]:
# load scalp example
import trimesh
import numpy as np
import pyvista as pv
from electrodes_positions.utils.mesh_utils import faces_to_pyvista

# scalp surface is from subject 16, session 2 in
# Telesford, Q.K., Gonzalez-Moreira, E., Xu, T. et al. An open-access dataset of naturalistic viewing using simultaneous EEG-fMRI. Sci Data 10, 554 (2023). https://doi.org/10.1038/s41597-023-02458-8

mesh = trimesh.load('outer_skin.stl')
vertices = np.array(mesh.vertices)
faces = np.array(mesh.faces)

# Picking points utils
## pick fiducials
With this utility you can choose the four fiducial landmarks on the subject's head

In [2]:
from electrodes_positions.utils.point_picking import pick_fiducials

# pick fiducials using a GUI
picked_points = pick_fiducials(vertices, faces)

Widget(value='<iframe src="http://localhost:39507/index.html?ui=P_0x7fce483f78f0_0&reconnect=auto" class="pyvi…

In [3]:
from electrodes_positions.utils.point_picking import project_fid_on_mesh

# project the fiducials on the mesh vertices
(RPA, LPA, NAS, IN), (RPA_idx, LPA_idx, NAS_idx, IN_idx) = project_fid_on_mesh(picked_points, vertices, return_positions = True, return_indices=True)

## selects the three regions on which electrodes should not be placed
with this utils you can select closed paths on the head from which to remove missplaced electrodes (i.e. the ears and the eyes)

In [4]:
from electrodes_positions.utils.point_picking import pick_closed_path, select_feasible_positions

In [5]:
picked_points, paths = pick_closed_path(vertices, faces)

Widget(value='<iframe src="http://localhost:35481/index.html?ui=P_0x7fce6f26b170_0&reconnect=auto" class="pyvi…

In [6]:
right_ear = picked_points
picked_points, paths = pick_closed_path(vertices, faces)

Widget(value='<iframe src="http://localhost:35481/index.html?ui=P_0x7fce6f123ec0_0&reconnect=auto" class="pyvi…

In [7]:
left_ear = picked_points
picked_points, paths = pick_closed_path(vertices, faces)

Widget(value='<iframe src="http://localhost:35481/index.html?ui=P_0x7fce7ddd2480_0&reconnect=auto" class="pyvi…

In [8]:
eyes = picked_points

In [9]:
right_ear = vertices[right_ear]
left_ear = vertices[left_ear]
eyes = vertices[eyes]

# Montage builders

## Standard Montages

### 10-10

In [5]:
# creates a standard montage according to the desired system
# see available montages with print(electrodes_positions.montages.available_montages)
from electrodes_positions.montages import create_standard_montage

newverts, newfac, all_landmarks = create_standard_montage(vertices, faces, fiducials = (RPA_idx, LPA_idx, NAS_idx, IN_idx), system = '10-10', return_indices = True)

In [11]:
selected_landmarks = select_feasible_positions(newverts, newfac, outlines = [right_ear, left_ear, eyes], landmarks = all_landmarks, positions = None, project_outlines = True)

100%|██████████| 87/87 [00:00<00:00, 423.10it/s]


In [ ]:
mesh = pv.PolyData(newverts, faces_to_pyvista(newfac))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_point_labels(newverts[np.array(list(selected_landmarks.values()))], list(selected_landmarks.keys()), render_points_as_spheres = True, point_size = 10, point_color = 'yellow')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 20, point_color = 'green')
plotter.show()

Widget(value='<iframe src="http://localhost:41673/index.html?ui=P_0x7f3c8b533bf0_1&reconnect=auto" class="pyvi…

### 10-5

In [ ]:
# creates a standard montage according to the desired system
# see available montages with print(electrodes_positions.montages.available_montages)
from electrodes_positions.montages import create_standard_montage

newverts, newfac, all_landmarks = create_standard_montage(vertices, faces, fiducials = (RPA_idx, LPA_idx, NAS_idx, IN_idx), system = '10-5-full', return_indices = True)

In [27]:
selected_landmarks = select_feasible_positions(newverts, newfac, outlines = [right_ear, left_ear, eyes], landmarks = all_landmarks, positions = None, project_outlines = True)

100%|██████████| 345/345 [00:01<00:00, 223.55it/s]


In [ ]:
mesh = pv.PolyData(newverts, faces_to_pyvista(newfac))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_point_labels(newverts[np.array(list(selected_landmarks.values()))], list(selected_landmarks.keys()), render_points_as_spheres = True, point_size = 10, point_color = 'yellow')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 20, point_color = 'green')
plotter.show()

Widget(value='<iframe src="http://localhost:41673/index.html?ui=P_0x7f3c686d67e0_5&reconnect=auto" class="pyvi…

## Custom montage
A custom montage is defined as a further subdivision of the 10-10 system, in particular:
* subdivisions = 1 coincides with the 10-10 system
* subdivisions = 2 coincides with the 10-5 system
* subdivisions = 3 would be the 10-3.3 system
* subdivisions = 4 would be the 10-2.5 system

### 4 subdivisions, 10-2.5 system

In [ ]:
# creates a custom montage according to the desired subdivisions
from electrodes_positions.montages import create_custom_montage

newverts, newfac, all_pos, all_landmarks = create_custom_montage(vertices, faces, fiducials = (RPA_idx, LPA_idx, NAS_idx, IN_idx), subdivisions = 4, return_indices = True)

In [17]:
selected_pos = select_feasible_positions(newverts, newfac, outlines = [right_ear, left_ear, eyes], landmarks = all_landmarks, positions = all_pos, project_outlines = True)

100%|██████████| 1265/1265 [00:03<00:00, 416.69it/s]


In [ ]:
mesh = pv.PolyData(newverts, faces_to_pyvista(newfac))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_points(newverts[selected_pos], render_points_as_spheres = True, point_size = 10, color = 'yellow')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 20, point_color = 'green')
plotter.show()

Widget(value='<iframe src="http://localhost:41673/index.html?ui=P_0x7f3c68465190_7&reconnect=auto" class="pyvi…

### 8 subdivisions, 10-1.25 system

In [19]:
# creates a custom montage according to the desired subdivisions
from electrodes_positions.montages import create_custom_montage

newverts, newfac, all_pos, all_landmarks = create_custom_montage(vertices, faces, fiducials = (RPA_idx, LPA_idx, NAS_idx, IN_idx), subdivisions = 8, return_indices = True)

In [20]:
selected_pos = select_feasible_positions(newverts, newfac, outlines = [right_ear, left_ear, eyes], landmarks = all_landmarks, positions = all_pos, project_outlines = True)

100%|██████████| 4961/4961 [00:16<00:00, 304.95it/s]


In [21]:
mesh = pv.PolyData(newverts, faces_to_pyvista(newfac))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_points(newverts[selected_pos], render_points_as_spheres = True, point_size = 10, color = 'yellow')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 20, point_color = 'green')
plotter.show()

Widget(value='<iframe src="http://localhost:35481/index.html?ui=P_0x7fce645312b0_3&reconnect=auto" class="pyvi…

### 16 subdivisions, 10-0.625 system

In [25]:
# creates a custom montage according to the desired subdivisions
from electrodes_positions.montages import create_custom_montage

newverts, newfac, all_pos, all_landmarks = create_custom_montage(vertices, faces, fiducials = (RPA_idx, LPA_idx, NAS_idx, IN_idx), subdivisions = 16, return_indices = True)

In [29]:
selected_pos = select_feasible_positions(newverts, newfac, outlines = [right_ear, left_ear, eyes], landmarks = all_landmarks, positions = all_pos, project_outlines = True)

100%|██████████| 19649/19649 [02:38<00:00, 123.79it/s]


In [ ]:
mesh = pv.PolyData(newverts, faces_to_pyvista(newfac))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_points(newverts[selected_pos], render_points_as_spheres = True, point_size = 10, color = 'yellow')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 20, point_color = 'green')
plotter.show()

## Random montage
A random montage is a collection of electrodes positions sampled randomly above the last line of electrodes as defined in the 10-10 system. There are two possible sampling strategies:
* Poisson sampling: where positions are sampled to be not to close to each other
* Uniform sampling: where positions are sampled uniformly across the entire surface

### Poisson sampling: 100 electrodes
Poisson sampling is performed by specifying the minimum distance between all the electrodes. When a number of electrodes is chosen, an estimate is made on the minimum distance required to cover the head with the specified number of positions.

In [ ]:
# creates a random montage according to the desired number of electrodes or minimal distance
from electrodes_positions.montages import create_random_montage

newverts, newfac, all_pos, all_landmarks = create_random_montage(vertices, faces, fiducials = (RPA_idx, LPA_idx, NAS_idx, IN_idx), num_electrodes = 100, return_indices = True)

Performing Sampling on the mesh


Iteration 1111: : 1110it [00:51, 21.55it/s]                           


: 

In [ ]:
selected_pos = select_feasible_positions(newverts, newfac, outlines = [right_ear, left_ear, eyes], landmarks = all_landmarks, positions = all_pos, project_outlines = True)

100%|██████████| 19649/19649 [02:38<00:00, 123.79it/s]


In [ ]:
mesh = pv.PolyData(newverts, faces_to_pyvista(newfac))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_points(newverts[selected_pos], render_points_as_spheres = True, point_size = 10, color = 'yellow')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 20, point_color = 'green')
plotter.show()

### Uniform sampling: 100 electrodes
Uniform sampling is uniform wrt mesh area

In [ ]:
# creates a random montage according to the desired number of electrodes or minimal distance
from electrodes_positions.montages import create_random_montage

newverts, newfac, all_pos, all_landmarks = create_random_montage(vertices, faces, fiducials = (RPA_idx, LPA_idx, NAS_idx, IN_idx), sampling = 'uniform', num_electrodes = 100, return_indices = True)

Performing Sampling on the mesh
Projecting sampled points on original mesh


In [ ]:
selected_pos = select_feasible_positions(newverts, newfac, outlines = [right_ear, left_ear, eyes], landmarks = all_landmarks, positions = all_pos, project_outlines = True)

100%|██████████| 19649/19649 [02:38<00:00, 123.79it/s]


In [ ]:
mesh = pv.PolyData(newverts, faces_to_pyvista(newfac))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_points(newverts[selected_pos], render_points_as_spheres = True, point_size = 10, color = 'yellow')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 20, point_color = 'green')
plotter.show()

Widget(value='<iframe src="http://localhost:39507/index.html?ui=P_0x7fce20605730_2&reconnect=auto" class="pyvi…